# Lexical parallels for parasha #32: Behar (Leviticus 25:1-26:2)

## Table of Content<a class="anchor" id="TOC"></a> (ToC)

* <a href="#bullet1">1 - Introduction</a>
* <a href="#bullet2">2 - Load Text-Fabric app and data</a>
* <a href="#bullet3">3 - Performing the queries</a>
    * <a href="#bullet3x1">3.1 - Locate the parallels</a>
* <a href="#bullet4">4 - Required libraries</a>
* <a href="#bullet5">5 - Further reading</a>
* <a href="#bullet6">6 - Notebook version details</a>

# 1 - Introduction <a class="anchor" id="bullet1"></a>
##### [Back to ToC](#TOC)

In this notebook we search for lexical parallels between verses in this parasha with other verses in the Tenach.

# 2 - Load Text-Fabric app and data <a class="anchor" id="bullet2"></a>
##### [Back to ToC](#TOC)

The following code will load the Text-Fabric version of the [Biblia Hebraica Stuttgartensia (Amstelodamensis)](https://etcbc.github.io/bhsa/).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Loading the Text-Fabric code
# Note: it is assumed Text-Fabric is installed in your environment.
from tf.fabric import Fabric
from tf.app import use

In [3]:
# load the app and data
BHSA = use ("etcbc/BHSA", mod="tonyjurg/BHSaddons/tf/", hoist=globals())

**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


# 3 - Performing the queries <a class="anchor" id="bullet3"></a>
##### [Back to ToC](#TOC)

The main engine of our queries is the use of Text-Fabric feature `crossref`, part of `Parallel Passages` module. See also [this notebook](https://nbviewer.org/github/etcbc/parallels/blob/master/programs/parallels.ipynb) explaing the concepts and how this feature was created.

## 3.1 - Locate the parallels <a class="anchor" id="bullet3x1"></a>

In [4]:
# find all verse nodes for this parasha using its sequence number
parashaQuery = '''
verse parashanum=32
'''
parashaResults = BHSA.search(parashaQuery)

  0.02s 57 results


In [5]:
# Store parashname, start and end verse for future use
startNode=parashaResults[0][0]
endNode=parashaResults[-1][0]
parashaNameHebrew=F.parashahebr.v(startNode)
parashaNameEnglish=F.parashatrans.v(startNode)
bookStart,chapterStart,startVerse=T.sectionFromNode(startNode)
parashaStart=f'{bookStart} {chapterStart}:{startVerse}'
bookEnd,chapterEnd,startEnd=T.sectionFromNode(endNode)
parashaEnd=f'{chapterEnd}:{startEnd}'
htmlStart='<html><body>'
htmlFooter=f'<p>Data generated by `hapax.ipynb` at `<a href=\"https://github.com/tonyjurg/Parashot\" target=\"_blank\">github.com/tonyjurg/Parashot</a>`</p></body></html>'

In [6]:
from difflib import SequenceMatcher
from IPython.display import HTML, display

# Function to find and highlight matching parts between two strings
def highlightMatches(baseText, comparisonText):
    matcher = SequenceMatcher(None, baseText, comparisonText)
    highlightedComparisonText = ""
    
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == "equal":  # Identical parts
            highlightedComparisonText += f"<mark>{comparisonText[j1:j2]}</mark>"
        else:  # Non-matching parts
            highlightedComparisonText += comparisonText[j1:j2]
    
    return highlightedComparisonText

# Function to process cross-references and format them into an HTML table
def generateCrossReferencesTable(verseNode):
    """
    Generates an HTML table with cross-references for a single verse node, highlighting identical parts.
    The main verse text will be right-aligned.
    """
    # Get cross-references for the specified verseNode
    crossRefs = Es("crossref").f(verseNode)
    tableContent = ""
    
    # Check if there are any cross-references for this verse
    if crossRefs:
        verseSection = T.sectionFromNode(verseNode)
        mainVerseText = T.text(verseNode)
        linkStepBible = (
            f"<a href=\"https://www.stepbible.org/?q=version=NASB2020%7Creference={verseSection[0]}.{verseSection[1]}:{verseSection[2]}&options=HNVUG\" target=\"_blank\">"
            f"{verseSection[0]} {verseSection[1]}:{verseSection[2]}</a>"
        ) 
        # Right-align the main verse text
        tableContent += f"<h4>Cross-references for {linkStepBible}</h4>"
        tableContent += f"<div style='text-align: right; font-weight: bold;'>{mainVerseText}</div>"
        # Create table header
        tableContent += f"<table border='1' style='border-collapse: collapse; width: 100%;'><tr><th>Reference</th><th>Match</th><th>Text</th></tr>"
        
        # Process each cross-reference and add a row to the table
        for target, confidence in crossRefs:
            targetSection = T.sectionFromNode(target)
            targetText = T.text(target)
            
            targetStepBible = (
                f"<a href=\"https://www.stepbible.org/?q=version=NASB2020%7Creference={targetSection[0]}.{targetSection[1]}:{targetSection[2]}&options=HNVUG\" target=\"_blank\">"
                f"{targetSection[0]} {targetSection[1]}:{targetSection[2]}</a>"
            )     
            # Highlight identical parts in target verse
            highlightedText = highlightMatches(mainVerseText, targetText)
            
            # Add the row for the cross-reference
            tableContent += f"<tr><td>{targetStepBible}</td><td>{confidence}%</td><td>{highlightedText}</td></tr>"
        
        # Close the table
        tableContent += "</table><br>"

    return tableContent

# Initialize HTML content
htmlContent = f"<h2>Lexical parallels for parasha {parashaNameEnglish} ({parashaStart}-{parashaEnd})</h2>"

# Process each verse and generate cross-reference tables
for verse in parashaResults:
    htmlContent += generateCrossReferencesTable(verse[0])

# Save the content to an HTML file
fileName = f"lexical_parallels({parashaNameEnglish.replace(' ','%20')}).html"
with open(fileName, "w", encoding="utf-8") as file:
    file.write(htmlContent)

# Display the HTML content in the notebook
display(HTML(htmlContent))

# wrap html header and footer and display a download button
htmlContentFull = f'{htmlStart}{htmlContent}{htmlFooter}'
downloadButton = f"""
<a download="{fileName}" href="data:text/html;charset=utf-8,{htmlContent.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;').replace('"', '&quot;').replace("'", '&#39;')}" target="_blank">
    <button>Download Lexical Parallels as HTML</button>
</a>
"""
display(HTML(downloadButton))

Reference,Match,Text
Exodus 6:10,81%,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃
Exodus 7:8,85%,וַיֹּ֣אמֶר יְהוָ֔ה אֶל־מֹשֶׁ֥ה וְאֶֽל־אַהֲרֹ֖ן לֵאמֹֽר׃
Exodus 13:1,81%,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃
Exodus 14:1,81%,וַיְדַבֵּ֥ר יְהֹוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃
Exodus 16:11,81%,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃
Exodus 25:1,81%,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃
Exodus 30:11,81%,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃
Exodus 30:17,81%,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃
Exodus 30:22,81%,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃
Exodus 31:1,81%,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃


# 4 - Required libraries <a class="anchor" id="bullet4"></a>
##### [Back to ToC](#TOC)

The scripts in this notebook require (beside `text-fabric`) the following Python libraries to be installed in the environment:

    difflib

You can install any missing library from within Jupyter Notebook using either`pip` or `pip3`.

# 5 - Further reading <a class="anchor" id="bullet5"></a>
##### [Back to ToC](#TOC)

For an elaborate treatment of parallel passages, see:

> Willem Th. van Peursen and Eep Talstra. "Computer-Assisted Analysis of Parallel Texts in the Bible - The Case of 2 Kings xviii-xix and its Parallels in Isaiah and Chronicles" in *Vetus Testamentum* 57, pp. 45-72. 2007, Brill, Leiden.

# 6 - Notebook version details<a class="anchor" id="bullet6"></a>
##### [Back to ToC](#TOC)

<div style="float: left;">
  <table>
    <tr>
      <td><strong>Author</strong></td>
      <td>Tony Jurg</td>
    </tr>
    <tr>
      <td><strong>Version</strong></td>
      <td>1.1</td>
    </tr>
    <tr>
      <td><strong>Date</strong></td>
      <td>5 March 2025</td>
    </tr>
  </table>
</div>